In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 59.8 MB/s 


In [ ]:
import statsmodels.api as sm
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('train_dataset_train.csv')
df.head()

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
0,1,40BD89EC85646EFB69E283F39C298E60,Пропуск FacePay,2402,Лефортово БКЛ,11007,Лефортово,11,Большая кольцевая,2022-09-12 05:00:13,216.316667,8001.0
1,2,126727A96489CC976A8C08E5CEB00542,СК учащегося 30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:54,648.183333,9011.0
2,3,D28CE6A9E0E5B6D213470A97CFF32485,БСК дружинника г.Москвы,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:00:55,865.333333,7022.0
3,4,015DA44B523C062B5BFEFF3FB0E64B9E,30 дней,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:01:13,1048.233333,2022.0
4,5,95B19C6F3A504727AC3EA56EB7E3E80F,КОШЕЛЕК,110,Войковская ( Южный ),2006,Войковская,2,Замоскворецкая,2022-09-12 05:02:55,965.600000,2017.0


In [ ]:
df.describe(include='all')

,id,ticket_id,ticket_type_nm,entrance_id,entrance_nm,station_id,station_nm,line_id,line_nm,pass_dttm,time_to_under,label
count,14606,14606,14606,14605,14605,14605,14605,14605,14605,14604,14604.000000,14603.000000
unique,14606,14606,48,411,411,271,241,17,17,3977,NaN,NaN
top,1,40BD89EC85646EFB69E283F39C298E60,30 дней ЕДИНЫЙ ТК,153,Щёлковская ( Северный ),3022,Щёлковская,7,Таганско-Краснопресненская,2022-09-12 06:51:22,NaN,NaN
freq,1,1,3921,292,292,356,356,2153,2153,15,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,701.484044,8727.394987
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,177.872010,13917.397915
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.283333,1002.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,635.595833,3008.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736.100000,6020.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800.687500,9025.000000


In [ ]:
df.drop(columns=['entrance_id','station_id','line_id','id'],inplace=True)

In [ ]:
df.isnull().sum()

ticket_id         0
ticket_type_nm    0
entrance_nm       1
station_nm        1
line_nm           1
pass_dttm         2
time_to_under     2
label             3
dtype: int64

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.drop(columns=['ticket_id'], inplace=True)

In [ ]:
def time_preprocess_r(df):
    df['day'] = df.pass_dttm.dt.dayofweek #день недели
    df['hour'] = df.pass_dttm.dt.hour # час в формате 24
    df['workday'] = df['day'].apply(lambda x: 0 if x == 5 or x == 6 else 1)
    df = df.drop(columns=['pass_dttm','entrance_nm','station_nm','line_nm','ticket_type_nm','label'])
    return df

In [ ]:
df.pass_dttm = pd.to_datetime(df.pass_dttm)
df = sm.add_constant(df)
x_regress_train, x_regress_test, y_regress_train, y_regress_test = train_test_split(df.drop(columns=['time_to_under']), df[['time_to_under']], test_size=0.3)
x_regress_train = time_preprocess_r(x_regress_train)
x_regress_test = time_preprocess_r(x_regress_test)

model_regression = sm.OLS(y_regress_train,x_regress_train)
model_regression=model_regression.fit()
forecast_regression= model_regression.predict(x_regress_test)

/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
def time_preprocess_c(df):
    df['day'] = df.pass_dttm.dt.dayofweek #день недели
    df['hour'] = df.pass_dttm.dt.hour # час в формате 24
    df['workday'] = df['day'].apply(lambda x: 0 if x == 5 or x == 6 else 1)

    df['name_workday'] = df['day'].apply(lambda x : 'holiday' if x == 5 or x ==6  else 'work')

    df = df.drop(columns=['pass_dttm','day','hour','workday'])
    return df

In [ ]:
df.pass_dttm = pd.to_datetime(df.pass_dttm)

x_class_train = df.drop(columns=['label'])[:400]
y_class_train = df[['label']][:400]


test_class = df.loc[df.ticket_type_nm.isin(x_class_train.ticket_type_nm)&
                    df.entrance_nm.isin(x_class_train.entrance_nm) &
                    df.station_nm.isin(x_class_train.station_nm) &
                    df.line_nm.isin(x_class_train.line_nm) ]

x_class_test = test_class.drop(columns=['label'])[-150:]
y_class_test = test_class[['label']][-150:]

x_class_train = time_preprocess_c(x_class_train)
x_class_test = time_preprocess_c(x_class_test)


model_class = CatBoostClassifier(iterations = 10)
model_class.fit(x_class_train,y_class_train, cat_features=['name_workday','ticket_type_nm','entrance_nm','station_nm','line_nm'])
forecast_class= model_class.predict(x_class_test)


Learning rate set to 0.5
0:	learn: 5.0415292	total: 4.52s	remaining: 40.7s
1:	learn: 4.8932751	total: 8.95s	remaining: 35.8s
2:	learn: 4.7664303	total: 13.4s	remaining: 31.3s
3:	learn: 4.6304619	total: 17.9s	remaining: 26.9s
4:	learn: 4.5393623	total: 22.4s	remaining: 22.4s
5:	learn: 4.3983501	total: 29.1s	remaining: 19.4s
6:	learn: 4.3017775	total: 34.6s	remaining: 14.8s
7:	learn: 4.1676358	total: 39.1s	remaining: 9.78s
8:	learn: 4.0287492	total: 43.6s	remaining: 4.84s
9:	learn: 3.8976746	total: 48.1s	remaining: 0us


Проверим точность

In [ ]:
from sklearn.metrics import r2_score, recall_score, mean_absolute_percentage_error
def result(actual_class, forecast_class, actual_regress, forecast_regress):
    return 0.5 * r2_score(actual_regress, forecast_regress) + 0.5 * recall_score(actual_class, forecast_class, average='micro')


In [ ]:
print('result: ', result(y_class_test, forecast_class, y_regress_test, forecast_regression))

result:  0.013261975619738879
